In [1]:
from PandaStation import *
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import subprocess


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp, Quaternion,
    RandomGenerator, UniformlyRandomRotationMatrix
    )
import pydrake.perception as mut
import open3d as o3d
from ompl import base as ob
from ompl import geometric as og
import time
from enum import Enum

ycb = {"cracker": "drake/manipulation/models/ycb/sdf/003_cracker_box.sdf", 
    "sugar": "drake/manipulation/models/ycb/sdf/004_sugar_box.sdf", 
    "soup": "drake/manipulation/models/ycb/sdf/005_tomato_soup_can.sdf", 
    "mustard": "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf", 
    "gelatin": "drake/manipulation/models/ycb/sdf/009_gelatin_box.sdf", 
    "meat": "drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf",
    "brick": "drake/examples/manipulation_station/models/061_foam_brick.sdf"}

for key in ycb.keys():
    ycb[key] = FindResourceOrThrow(ycb[key])
    
ycb["puck"] = FindResource("models/puck.urdf")
ycb["marble"] = FindResource("models/marble.urdf")

def ycb_resource(name):
    global ycb
    return ycb[name]

def random_ycb_resource():
    global ycb
    ycb_items = list(ycb.items())
    index = np.random.randint(0, len(ycb_items))
    return ycb_resource(ycb_items[index][0])

In [2]:
class Stage(Enum):
    INIT = 0
    TO_PREPICK = 1
    TO_PICK = 2
    PICKING = 3
    TO_POSTPICK = 4
    TO_INIT = 5
  
class FindGrasp(LeafSystem):
    
    def __init__(self, station, station_context):
        LeafSystem.__init__(self)
        self.nq = 6
        self.omni_grasp_position_output_port = self.DeclareVectorOutputPort(
                                                                    "omni_grasp_position", 
                                                                    BasicVector(self.nq),
                                                                    self.DoCalcOutput)
        self.station = station
        self.station_context = station_context
        self.rng = np.random.default_rng()
          
    def DoCalcOutput(self, context, output):
        start_time = time.time()
        ik_station, body_infos = create_welded_station(self.station, self.station_context, omni = True)
        ik_station_context = ik_station.CreateDefaultContext()
        q, cost = grasp_pose(body_infos[0], ik_station, ik_station_context, q_nominal = [0.6, 0, 0.3, 0, np.pi, 0])
        assert np.isfinite(cost), "No grasp could be found"
        output.set_value(q)

class Picker(LeafSystem):
    
    def __init__(self, station, station_context):
        LeafSystem.__init__(self)
        self.nq = 6
        self.grasp_position_input_port = self.DeclareVectorInputPort("grasp_position", BasicVector(self.nq))
        self.omni_position_input_port = self.DeclareVectorInputPort("omni_position", 
                BasicVector(self.nq))
        self.omni_position_command_output_port = self.DeclareVectorOutputPort(
                "omni_position_command", BasicVector(self.nq),
                self.DoCalcOutput)
        self.hand_command_output_port = self.DeclareVectorOutputPort("hand_position_command", BasicVector(1),
                                                                    self.CalcHandOutput)
        self.station = station
        self.station_context = station_context
        self.plant = station.get_multibody_plant()
        self.omni = self.plant.GetModelInstanceByName("omni_arm")
        self.hand = self.plant.GetModelInstanceByName("hand")
        self.plant_context = self.plant.GetMyContextFromRoot(self.station_context)
        self.scene_graph = station.get_scene_graph()
        self.scene_graph_context = station.GetSubsystemContext(self.scene_graph, self.station_context)
        self.query_output_port = self.scene_graph.GetOutputPort("query")
        self.q_S = None
        self.q_G = None
        self.q_P = None
        self.omni_traj = None
        self.hand_traj = None
        self.X_Pregrasp = None
        self.X_Grasp = None
        self.stage = Stage.INIT 
        self.setup_time = 0.5
        self.p_tol = 0.01*np.ones(3)
        self.theta_tol = 0.05
        
    def CalcHandOutput(self, context, output):
        time = context.get_time()
        if (self.hand_traj is None) or (time > self.hand_traj.end_time()):
            output.set_value([0])
        else:
            output.set_value(self.hand_traj.value(time).flatten())
        
    def DoCalcOutput(self, context, output):
        time = context.get_time()
        if time < self.setup_time: # allow time for objects to settle
            self.q_S = self.EvalVectorInput(context, self.omni_position_input_port.get_index()).get_value()
            output.set_value(self.q_S)
            return
        
        if self.stage == Stage.INIT:
            print("INITIALIZATION")
            self.q_G = self.EvalVectorInput(context, self.grasp_position_input_port.get_index()).get_value()
            self.q_P, self.X_Pregrasp, self.X_Grasp = self.pregrasp_q()
            self.omni_traj = self.default_rrt_trajectory(self.q_S, time, self.q_P, time + 3)
            self.hand_traj = self.first_order_traj([0.0], time, [0.08], time + 3)
            self.stage = Stage.TO_PREPICK
            print("TO PREPICK")
            
        if self.stage == Stage.TO_PREPICK:
            
            if time <= self.omni_traj.end_time():
                output.set_value(self.omni_traj.value(time).flatten())
            else:
                self.stage = Stage.TO_PICK
                print("TO PICK")
                self.omni_traj = self.default_rrt_trajectory(self.q_P, time, self.q_G, time + 2)
                self.hand_traj = self.first_order_traj([0.08], time, [0.08], time +2)
                
        if self.stage == Stage.TO_PICK:
            if time <= self.omni_traj.end_time():
                output.set_value(self.omni_traj.value(time).flatten())
            else:
                self.stage = Stage.PICKING
                print("PICKING")
                self.hand_traj = self.first_order_traj([0.08], time, [0.0], time +2)
                self.omni_traj = self.first_order_traj(self.q_G, time, self.q_G, time +2)
        
        if self.stage == Stage.PICKING:
            if time <= self.omni_traj.end_time():
                output.set_value(self.omni_traj.value(time).flatten())
            else:
                self.stage = Stage.TO_POSTPICK
                print("TO POSTPICK")
                self.omni_traj = self.default_rrt_trajectory(self.q_G, time, self.q_S, time + 3)
                self.hand_traj = self.first_order_traj([0.0], time, [0.0], time + 3)
                
        if self.stage == Stage.TO_POSTPICK:
            if time <= self.omni_traj.end_time():
                output.set_value(self.omni_traj.value(time).flatten())
            else:
                pass
                 
    def straight_line_traj(self, q_start, X_start, start_time, X_end, end_time, N = 10):
        # interpolate between X_start and X_end (we want to maintain same gripper oritentation)
        times = np.array([start_time, end_time])
        ps = np.array([X_start.translation(), X_end.translation()])
        p_traj = PiecewisePolynomial.FirstOrderHold(times, ps.T)
        times = np.linspace(start_time, end_time, N)
        qs = []
        R_WG = X_end.rotation()
        for i,t in enumerate(times):
            p_WG = p_traj.value(t)
            X_WG = RigidTransform(R_WG, p_WG)
            if i == 0:
                warm_start = q_start
            else:
                warm_start = qs[-1]
            q = self.X_WG_to_q(X_WG, warm_start = warm_start)
            qs.append(q)
            
        qs = np.array(qs)
        return PiecewisePolynomial.CubicShapePreserving(times, qs.T)
                
    def first_order_traj(self, q_start, start_time, q_end, end_time):
        time = np.array([start_time, end_time])
        qs = np.array([q_start, q_end])
        return PiecewisePolynomial.FirstOrderHold(time, qs.T)
            
    def default_rrt_trajectory(self, q_start, start_time, q_goal, goal_time):
        ik_station, _ = create_welded_station(self.station, self.station_context, omni = True)
        plant = ik_station.get_multibody_plant()
        ik_station_context = ik_station.CreateDefaultContext()
        plant_context = ik_station.GetSubsystemContext(plant, ik_station_context)
        omni = plant.GetModelInstanceByName("omni_arm")
        hand = plant.GetModelInstanceByName("hand")
        scene_graph = ik_station.get_scene_graph()
        scene_graph_context = ik_station.GetSubsystemContext(scene_graph, ik_station_context)
        query_output_port = scene_graph.GetOutputPort("query")
        
        def is_colliding(q):
            #plant.SetPositions(plant_context, omni, q)
            #query_object = query_output_port.Eval(scene_graph_context)
            return False #query_object.HasCollisions()
        
        def isStateValid(state):
            #q = self.parse_state(state)
            return True #not is_colliding(q)

        joint_limits = self.joint_limits()
        space = ob.SE3StateSpace()
        bounds = ob.RealVectorBounds(3)
        for i in range(3):
            bounds.setLow(i, joint_limits[i][0])
            bounds.setHigh(i, joint_limits[i][1])
            
        space.setBounds(bounds)
        ss = og.SimpleSetup(space)
        ss.setStateValidityChecker(ob.StateValidityCheckerFn(isStateValid))
        start = self.q_to_state(space, q_start)
        print(start)
        assert isStateValid(start), "you have supplied an invalid start state RRT"
        goal = self.q_to_state(space, q_goal)
        print(goal)
        assert isStateValid(goal), "you have supplied an invalid goal state for RRT"
        ss.setStartAndGoalStates(start, goal) 
        print("starting to solve")
        return None
        solved = ss.solve()
        assert solved
        ss.simplifySolution()
        path = ss.getSolutionPath()
        res = []
        costs = []
        for state in path.getStates():
            q = self.parse_state(state)
            if len(costs) == 0:
                costs.append(0)
            else:
                d = q - res[-1]
                cost = np.sqrt(d.dot(d))
                costs.append(costs[-1] + cost)
            res.append(q)
        
        if (costs[-1] == 0):
            costs[-1] = 1

        qs = np.array(res)
        times = start_time + ((goal_time - start_time)*np.array(costs)/costs[-1])
        if len(qs) < 3:
            return PiecewisePolynomial.FirstOrderHold(times, 
                    qs[:, 0:self.nq].T)
        
        return PiecewisePolynomial.CubicShapePreserving(times,
                qs[:, 0:self.nq].T)
    
    def joint_limits(self):
        joint_inds = self.plant.GetJointIndices(self.omni)[:self.nq]
        joint_limits = []
        for i in joint_inds:
            joint = self.plant.get_joint(i)
            joint_limits.append((joint.position_lower_limits()[0],
                                 joint.position_upper_limits()[0]))
        return joint_limits

    def parse_state(self, state):
        """ parses ompl RealVectorStateSpace::StateType into a numpy array"""
        '''
        q = np.array([state.getX(), state.getY(), state.getZ()])
        quat = [state.rotation().x, 
               state.rotation().y,
               state.rotation().z,
               state.rotation().w]
        '''
        print(state)
        q = np.array([state[0], state[1], state[2]])
        quat = [state[3], state[4], state[5], state[6]]
        rpy = RollPitchYaw(Quaternion(quat)).vector()
        q = np.concatenate((q, rpy))
        return q

    def q_to_state(self, space, q):
        """ turns a python list q into an ompl state"""
        state = ob.State(space)
        rpy = RollPitchYaw(q[3:])
        quat = rpy.ToQuaternion()
        rot = np.array([quat.x(), quat.y(), quat.z(), quat.w()])
        q = np.concatenate((q[:3], rot))
        for i in range(len(q)):
            state[i] = q[i]
        return state
            
    def pregrasp_q(self):
        ''' find a good pregrasp pose (self.q_P) given the grasping pose '''
        assert self.q_G is not None
        # find the current grasping pose
        X_Grasp = self.q_to_X_WG(self.q_G)
        
        # retreat in the negative z direction of the gripper frame
        X_Pregrasp = X_Grasp.multiply(RigidTransform(RotationMatrix(), [0, 0, -0.07]))
        q = self.X_WG_to_q(X_Pregrasp)

        return q, X_Pregrasp, X_Grasp
        
    def q_to_X_WG(self, q):
        assert len(q) == self.nq
        
        self.plant.SetPositions(self.plant_context, self.omni, q)   
        body = self.plant.GetBodyByName("panda_hand")
        return self.plant.EvalBodyPoseInWorld(self.plant_context, body)
        
    def X_WG_to_q(self, X_WG):
        
        translation = X_WG.translation()
        rpy = RollPitchYaw(X_WG.rotation())
        rpy = rpy.vector()
        res = np.concatenate((translation, rpy))
        
        return res
        
        """
        ik_station, _ = create_welded_station(self.station, self.station_context, omni = True)
        plant = ik_station.get_multibody_plant()
        plant_context = ik_station.GetSubsystemContext(plant, ik_station.CreateDefaultContext())
        ik = InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(
                plant.GetFrameByName("panda_hand"),
                np.zeros(3),
                plant.world_frame(),
                X_WG.translation() - self.p_tol,
                X_WG.translation() + self.p_tol)
        ik.AddOrientationConstraint(
                plant.GetFrameByName("panda_hand"),
                RotationMatrix(),
                plant.world_frame(),
                X_WG.rotation(),
                self.theta_tol)
        ik.AddMinimumDistanceConstraint(0.001, 0.1)
        q = ik.q()
        prog = ik.prog()
        prog.SetInitialGuess(q[0:2], X_WG.translation())
        rpy = RollPitchYaw(X_WG.rotation())
        prog.SetInitialGuess(q[3:], rpy.vector())
        result = Solve(prog)
        if not result.is_success():
            print("failed pregrasp ik, returning best result")
        return result.GetSolution(q)
        """

def random_position():
    return [0.6 + np.random.uniform(-0.05, 0.05), np.random.uniform(-0.1,0.1), 0.15]

def random_orientation():
    rot = RotationMatrix.MakeZRotation(np.random.uniform(-np.pi, np.pi))
    rot = rot.multiply(RotationMatrix.MakeXRotation(-np.pi/2))
    return rot

In [3]:
builder = pydrake.systems.framework.DiagramBuilder()
station = builder.AddSystem(OmniStation(time_step = 0.001))
station.SetupTableStation()


rs = np.random.RandomState()  # this is for python
generator = RandomGenerator(rs.randint(1000))  # this is for c++
    
item = "brick"
#station.AddModelFromFile(ycb_resource(item), 
#                    RigidTransform(random_orientation(), 
#                                    random_position()))

station.AddModelFromFile(ycb_resource(item),
                         RigidTransform(RotationMatrix(), [0.6, 0, 0.05]))


plant = station.get_multibody_plant()
omni = station.GetOmni()
station.Finalize()
station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
grasp_finder = builder.AddSystem(FindGrasp(station, station_context))
picker = builder.AddSystem(Picker(station, station_context))
builder.Connect(grasp_finder.GetOutputPort("omni_grasp_position"), picker.GetInputPort("grasp_position"))
builder.Connect(station.GetOutputPort("omni_position_measured"), picker.GetInputPort("omni_position"))
builder.Connect(picker.GetOutputPort("omni_position_command"), station.GetInputPort("omni_position"))
builder.Connect(picker.GetOutputPort("hand_position_command"), station.GetInputPort("hand_position"))
diagram = builder.Build()
simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
grasp_finder.station_context = station_context
picker.station_context = station_context
meshcat.start_recording()
simulator.AdvanceTo(10.5)
meshcat.stop_recording()
meshcat.publish_recording()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6015...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7015/static/
Connected to meshcat-server.
INITIALIZATION
Compound state [
RealVectorState [0.5 4.56308e-18 0.5]
SO3State [8.7864e-18 1 6.16069e-17 6.12323e-17]
]

Compound state [
RealVectorState [0.6 3.18078e-18 0.269493]
SO3State [-6.12323e-17 1 -6.12323e-17 1.61554e-15]
]

starting to solve
TO PREPICK


AttributeError: 'NoneType' object has no attribute 'end_time'

In [ ]:
"""
mustard
brick
marble
puck
soup
sugar
gelatin
meat
"""
#file = open(item + ".html", "w")
#file.write(meshcat.vis.static_html())
#file.close()

#print(subprocess.check_output(["tar", "vczf", "media/videos/ycb_pick/" + item + ".gz", item + ".html"]))
#print(subprocess.check_output(["trash", item +".html"]))